In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC, OneClassSVM
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

In [2]:
data_path = "./data/GEARBOX_processed.csv"
df = pd.read_csv(data_path)
df.head()

,Turbine_ID,Timestamp,Gear_Oil_Temp_Avg,Gear_Bear_Temp_Avg,Min_Windspeed1,Max_Windspeed1,Avg_Windspeed1,Var_Windspeed1,Min_Windspeed2,Max_Windspeed2,...,Anemometer2_CorrOffset,DistanceAirPress,AirRessureSensorZeroOffset,Anemometer1_Avg_Freq,Anemometer2_Avg_Freq,Pressure_Avg_Freq,Label,Next_Default_Date,Lead_Time,Default_in_60
0,T01,2016-01-01 00:00:00+00:00,44,48,3.7,6.0,5.1,0.21,3.8,6.0,...,0.0,0.0,600.0,98.0,99.0,418.0,0.0,2016-07-18 02:10:00+00:00,199,False
1,T01,2016-01-01 00:10:00+00:00,44,48,4.1,6.0,5.1,0.09,4.1,6.0,...,0.0,0.0,600.0,99.0,101.0,418.0,0.0,2016-07-18 02:10:00+00:00,199,False
2,T01,2016-01-01 00:20:00+00:00,43,46,4.5,6.7,5.7,0.26,4.4,6.8,...,0.0,0.0,600.0,111.0,113.0,418.0,0.0,2016-07-18 02:10:00+00:00,199,False
3,T01,2016-01-01 00:30:00+00:00,44,48,5.1,7.0,6.3,0.11,5.1,7.1,...,0.0,0.0,600.0,122.0,125.0,418.0,0.0,2016-07-18 02:10:00+00:00,199,False
4,T01,2016-01-01 00:40:00+00:00,44,48,4.7,7.3,6.2,0.27,4.9,7.4,...,0.0,0.0,600.0,121.0,123.0,417.0,0.0,2016-07-18 02:10:00+00:00,199,False


In [14]:
def get_data_with_fewer_features_onesvm(data_path= "./data/GEARBOX_processed.csv"):
    df = pd.read_csv(data_path)
    print(df.shape)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df['Next_Default_Date'] = pd.to_datetime(df['Next_Default_Date'])
    df['Label_SVM'] = df['Default_in_60'].astype(int)

    df.sort_values('Timestamp', inplace=True)
    window_size = 24*6
    df_features = df.drop(['Timestamp', 'Next_Default_Date', 'Label', 'Default_in_60', 'Lead_Time', 'Label_SVM'], axis=1)

    feat = df_features.iloc[:,1:]
    feat_var = feat.var().to_dict()
    feat_list = [c for c, var in feat_var.items() if abs(var-0) > 0.001]

    feat_list = [c for c in feat_list if "min" not in c.lower() and "max" not in c.lower()]

    corr_matrix = feat[feat_list].corr()
    corr_pairs = corr_matrix.unstack().reset_index()
    corr_pairs.columns = ['Variable1', 'Variable2', 'Correlation']
    # Removing self-correlation (diagonal)
    corr_pairs = corr_pairs[corr_pairs['Variable1'] != corr_pairs['Variable2']]
    # Removing duplicate pairs
    corr_pairs['sorted_row'] = [sorted([a, b]) for a, b in zip(corr_pairs['Variable1'], corr_pairs['Variable2'])]
    corr_pairs['sorted_row'] = corr_pairs['sorted_row'].astype(str)
    corr_pairs = corr_pairs.drop_duplicates(subset=['sorted_row'])
    corr_pairs.drop('sorted_row', axis=1, inplace=True)

    high_corr_pairs = corr_pairs[corr_pairs['Correlation'].abs() > 0.8]
    features_to_exclude = []
    for _, row in high_corr_pairs.iterrows():
        if row['Variable1'] not in features_to_exclude and row['Variable2'] not in features_to_exclude:
            # Strategy: Keep the first feature alphabetically
            feature_to_keep = min(row['Variable1'], row['Variable2'])
            feature_to_exclude = max(row['Variable1'], row['Variable2'])
            features_to_exclude.append(feature_to_exclude)

    feat_list = [c for c in feat_list if c not in features_to_exclude]

    df_features = df[['Turbine_ID'] + feat_list].copy()
    df_movin_mean = df_features.groupby('Turbine_ID').apply(lambda x: x.rolling(window=window_size, min_periods=1).mean())
    df_movin_mean.columns = [x+'_avg' for x in df_movin_mean.columns]
    df_movin_std = df_features.groupby('Turbine_ID').apply(lambda x: x.rolling(window=window_size, min_periods=1).std()).fillna(1)
    df_movin_std.columns = [x+'_std' for x in df_movin_std.columns]
    df = df[['Turbine_ID', 'Timestamp', 'Next_Default_Date', 'Label', 'Default_in_60', 'Lead_Time', 'Label_SVM']+ feat_list]
    df = pd.concat([df, df_movin_mean, df_movin_std], axis=1)
    
    test_size = 0.25
    val_size = 0.15

    df1 = df.sort_values(by='Timestamp')
    df1.reset_index(drop=True, inplace=True)
    first_label_1_index = df1[df1['Label'] == 1].index[0]
    len_df1 = len(df1)
    start_index = max(0, first_label_1_index - int(0.23 * len_df1))
    end_index = start_index + int(test_size * len_df1)

    test_data = df1.iloc[start_index:end_index].reset_index(drop=True)
    train_data = pd.concat([df1.iloc[:start_index], df1.iloc[end_index:]]).reset_index(drop=True)
    assert (len(df1) - (len(test_data) + len(train_data))==0)
    assert(len(test_data.Label.value_counts()) > 1 and len(train_data.Label.value_counts())>1)
    print(len(test_data), len(train_data))

    drop_ls = ['Turbine_ID', 'Timestamp', 'Next_Default_Date', 'Label', 'Default_in_60', 'Label_SVM', 'Lead_Time']

    X_balanced = train_data.drop(drop_ls, axis=1).copy()
    y_balanced = train_data['Label_SVM'].copy()
    print(X_balanced.shape, y_balanced.shape)
    # Split the balanced dataset into train and test sets
    X_train_balanced, X_test_balanced, y_train_balanced, y_test_balanced = train_test_split(X_balanced, y_balanced, test_size=val_size, random_state=42)

    test_set = test_data.drop(drop_ls, axis=1).copy()
    Y_event = test_data.apply(lambda x: x['Turbine_ID'] + ' ' + str(x['Next_Default_Date']), axis=1).tolist()
    Y_lead = test_data['Lead_Time'].tolist()

    return test_data, X_train_balanced, X_test_balanced, y_train_balanced, y_test_balanced, test_set, Y_event, Y_lead

In [3]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Next_Default_Date'] = pd.to_datetime(df['Next_Default_Date'])
df['Label_SVM'] = df['Default_in_60'].astype(int)

df.sort_values('Timestamp', inplace=True)
window_size = 24*6
df_features = df.drop(['Timestamp', 'Next_Default_Date', 'Label', 'Default_in_60', 'Lead_Time', 'Label_SVM'], axis=1)

df_movin_mean = df_features.groupby('Turbine_ID').apply(lambda x: x.rolling(window=window_size, min_periods=1).mean())
df_movin_mean.columns = [x+'_avg' for x in df_movin_mean.columns]
df_movin_std = df_features.groupby('Turbine_ID').apply(lambda x: x.rolling(window=window_size, min_periods=1).std()).fillna(1)
df_movin_std.columns = [x+'_std' for x in df_movin_std.columns]
df = pd.concat([df, df_movin_mean, df_movin_std], axis=1)
df.head()

,Turbine_ID,Timestamp,Gear_Oil_Temp_Avg,Gear_Bear_Temp_Avg,Min_Windspeed1,Max_Windspeed1,Avg_Windspeed1,Var_Windspeed1,Min_Windspeed2,Max_Windspeed2,...,Anemometer1_CorrOffset_std,Anemometer2_Freq_std,Anemometer2_Offset_std,Anemometer2_CorrGain_std,Anemometer2_CorrOffset_std,DistanceAirPress_std,AirRessureSensorZeroOffset_std,Anemometer1_Avg_Freq_std,Anemometer2_Avg_Freq_std,Pressure_Avg_Freq_std
0,T01,2016-01-01 00:00:00+00:00,44,48,3.7,6.0,5.1,0.21,3.8,6.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
172522,T07,2016-01-01 00:00:00+00:00,45,49,3.7,6.0,5.1,0.21,3.8,6.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
346873,T11,2016-01-01 00:00:00+00:00,48,55,3.7,6.0,5.1,0.21,3.8,6.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
87140,T06,2016-01-01 00:00:00+00:00,43,48,3.7,6.0,5.1,0.21,3.8,6.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
259723,T09,2016-01-01 00:00:00+00:00,43,48,3.7,6.0,5.1,0.21,3.8,6.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [4]:
test_size = 0.25
val_size = 0.15

df1 = df.sort_values(by='Timestamp')
df1.reset_index(drop=True, inplace=True)
first_label_1_index = df1[df1['Label'] == 1].index[0]
len_df1 = len(df1)
start_index = max(0, first_label_1_index - int(0.23 * len_df1))
end_index = start_index + int(test_size * len_df1)

test_data = df1.iloc[start_index:end_index].reset_index(drop=True)
train_data = pd.concat([df1.iloc[:start_index], df1.iloc[end_index:]]).reset_index(drop=True)
assert (len(df1) - (len(test_data) + len(train_data))==0)
assert(len(test_data.Label.value_counts()) > 1 and len(train_data.Label.value_counts())>1)
len(test_data), len(train_data)

(108528, 325587)

In [5]:
drop_ls = ['Turbine_ID', 'Timestamp', 'Next_Default_Date', 'Label', 'Default_in_60', 'Label_SVM', 'Lead_Time']

def balance_data(df):
    # Separate majority and minority classes
    df_majority = df[df.Label_SVM == 0].copy()
    df_minority = df[df.Label_SVM == 1].copy()

    # Downsample majority class
    df_majority_downsampled = resample(df_majority, 
                                    replace=False,    # sample without replacement
                                    n_samples=len(df_minority),     # to match minority class
                                    random_state=123) # reproducible results

    # Combine minority class with downsampled majority class
    df_balanced = pd.concat([df_majority_downsampled, df_minority])

    

    # Now let's extract the features and labels again from the balanced dataframe
    X_balanced = df_balanced.drop(drop_ls, axis=1).copy()
    # y_balanced = df_balanced['Label_SVM'].copy()
    y_balanced = df_balanced['Label'].copy()
    return X_balanced, y_balanced

# X_balanced, y_balanced = balance_data(train_data)
# Split the balanced dataset into train and test sets
df2 = train_data.sample(n=50000, random_state=42)
X_balanced = df2.drop(drop_ls, axis=1).copy()
y_balanced = df2['Label_SVM'].copy()
#y_balanced = train_data['Label'].copy()
X_train_balanced, X_test_balanced, y_train_balanced, y_test_balanced = train_test_split(X_balanced, y_balanced, test_size=val_size, random_state=42)

test_set = test_data.drop(drop_ls, axis=1).copy()
Y_event = test_data.apply(lambda x: x['Turbine_ID'] + ' ' + str(x['Next_Default_Date']), axis=1).tolist()
Y_lead = test_data['Lead_Time'].tolist()

In [6]:
def evaluate_y_pred(y_true, y_pred, Y_event, Y_lead):
    assert len(y_true) / len(y_pred) * len(Y_event) / len(Y_lead) == 1
        
    FP, FN, TP_lead = 0, 0, []
    warnings = {}
    for i in range(len(y_pred)):
        if y_true[i] == 0 and y_pred[i] == 1:
            FP += 1
        elif y_true[i] == 1:
            event = Y_event[i]
            lead_time = Y_lead[i]
            if event not in warnings and y_pred[i] == 0:
                warnings[event] = -1
            elif event not in warnings and y_pred[i] == 1:
                warnings[event] = lead_time
            elif event in warnings and warnings[event] < 0:
                if y_pred[i] == 1:
                    warnings[event] = lead_time
    for event, lead in warnings.items():
        if lead < 0:
            FN += 1
        else:
            TP_lead.append(lead)

    R, M, I = 100000, 20000, 5000
    savings = 0
    for l in TP_lead:
        savings += (l / 60) * (R - M)
    savings -= FP * I
    savings -= FN * R
    print(warnings.keys())
    return FP, FN, TP_lead, savings

In [7]:
def find_latest_consecutive_trues(arr, consec=2):
    latest_true_positions = []
    current_sequence_start = None
    current_sequence_length = 0

    for i, value in enumerate(arr):
        if value:
            current_sequence_length += 1
            if current_sequence_start is None:
                current_sequence_start = i
            if i == len(arr) - 1 and current_sequence_length >= consec:
                latest_true_positions.append(i)
        else:
            if current_sequence_start is not None and current_sequence_length >= consec:
                latest_true_positions.append(i-1)
            current_sequence_start = None
            current_sequence_length = 0

    return latest_true_positions

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, classification_report

nu = y_train_balanced.value_counts()[1] / len(y_train_balanced)
print(nu)

model = make_pipeline(StandardScaler(), OneClassSVM(kernel='rbf', nu=nu, max_iter=500000, ))

model.fit(X_train_balanced)
print('Training complete')
y_pred = model.predict(X_test_balanced)
# Convert predictions to a binary outcome for anomaly (1 for normal, -1 for anomaly) for both train and test
y_pred = (y_pred == -1)
# 评估结果
conf_matrix = confusion_matrix(y_test_balanced, y_pred)
class_report = classification_report(y_test_balanced, y_pred)

print(conf_matrix)
print(class_report)

0.026517647058823528
Training complete
[[7091  224]
 [ 185    0]]
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      7315
           1       0.00      0.00      0.00       185

    accuracy                           0.95      7500
   macro avg       0.49      0.48      0.49      7500
weighted avg       0.95      0.95      0.95      7500



In [15]:
test_data, X_train_balanced, X_test_balanced, y_train_balanced, y_test_balanced, test_set, Y_event, Y_lead = get_data_with_fewer_features_onesvm()

(434115, 48)
108528 325587
(325587, 27) (325587,)


In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, classification_report

nu = y_train_balanced.value_counts()[1] / len(y_train_balanced)
print(nu)

model = make_pipeline(StandardScaler(), OneClassSVM(kernel='rbf', nu=nu, max_iter=500000, ))

model.fit(X_train_balanced)
print('Training complete')
y_pred = model.predict(X_test_balanced)
# Convert predictions to a binary outcome for anomaly (1 for normal, -1 for anomaly) for both train and test
y_pred = (y_pred == -1)
# 评估结果
conf_matrix = confusion_matrix(y_test_balanced, y_pred)
class_report = classification_report(y_test_balanced, y_pred)

print(conf_matrix)
print(class_report)

0.02639946810817061
Training complete
[[46312  1253]
 [ 1262    12]]
              precision    recall  f1-score   support

           0       0.97      0.97      0.97     47565
           1       0.01      0.01      0.01      1274

    accuracy                           0.95     48839
   macro avg       0.49      0.49      0.49     48839
weighted avg       0.95      0.95      0.95     48839

